# 🚀 TensorFlow 数据效率优化：map 与 prefetch
## 1. 背景

在深度学习中，数据准备（加载、解析、增强）往往比模型训练更耗时。

如果不优化，GPU/TPU 可能会“等数据”，导致算力浪费。

- `tf.data.Dataset` 提供了 流水线式处理，其中：

  -` map()` 用来对数据集逐元素处理（如解析图片、增强数据）。

  - `prefetch()` 用来异步加载数据（边训练边准备下一批）。

## 2. map() 用法与参数详解

In [ ]:
dataset = dataset.map(
    map_func,
    num_parallel_calls=tf.data.AUTOTUNE
)


| 参数                   | 类型                       | 说明                               |
| -------------------- | ------------------------ | -------------------------------- |
| `map_func`           | function                 | 映射函数，定义每个元素如何被处理（如归一化、解码、增强等）。   |
| `num_parallel_calls` | int / `tf.data.AUTOTUNE` | 并行调用数。设置为 `AUTOTUNE` 可自动选择最优线程数。 |

### 示例：图片归一化

In [ ]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # 归一化
    return image, label

dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)


#### 📌 优势：

- 数据增强（翻转、旋转、裁剪等）可以放到 map 里。

- 并行处理大幅加快数据预处理。

---
## 3. prefetch() 用法与参数详解

In [ ]:
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)


| 参数            | 类型                       | 说明                                         |
| ------------- | ------------------------ | ------------------------------------------ |
| `buffer_size` | int / `tf.data.AUTOTUNE` | 预取的数据批次数。设置为 `AUTOTUNE` 时，TensorFlow 自动调优。 |

### 📌 作用：

 - 在 训练第 n 批数据时，后台会预先准备第 n+1 批数据。

 - 实现 计算（GPU/TPU）与数据准备（CPU）并行。

---
## 4. 综合示例：高效数据流水线

In [ ]:
import tensorflow as tf

# 假设 dataset 已经创建
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

dataset = (
    dataset
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)  # 并行预处理
    .shuffle(1000)                                         # 打乱数据
    .batch(32)                                             # 按批次分组
    .prefetch(tf.data.AUTOTUNE)                            # 异步加载
)


### ⚡ 优点

 - map()：高效的数据预处理（利用多核 CPU 并行）。

 - prefetch()：减少 GPU/TPU 等待数据的时间。

二者结合：最大化硬件利用率。

---
## 5. 总结

 - map()：逐元素预处理，可并行化 → CPU 加速。

 - prefetch()：异步预取批次 → GPU/TPU 不等待数据。

常见最佳实践：

In [ ]:
dataset = dataset.map(..., num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
